<h3>Temperature Anomaly Model - Designer</h3>
<img src="img/designer_model.png" width=600 />

In [1]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

SDK version: 1.12.0


In [2]:
from azureml.core import Workspace, Dataset

subscription_id = "68be6dca-bd5e-40f5-8590-32bbfd45bdd2"
resource_group = "sandbox-iot-aml-poc"
workspace_name = "sandbox-aml-iot-aml-poc"

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [3]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

model_name = 'tempanomaly'
package_output_directory = 'tempanomaly_container'

model = Model(workspace, model_name)
model.download(exist_ok=True)

designer_model_env = 'trained_model_outputs/conda_env.yaml'
designer_model_score = 'trained_model_outputs/score.py'

inference_config = InferenceConfig(entry_script=designer_model_score, conda_file=designer_model_env, runtime='python')

package = Model.package(workspace, [model], inference_config=inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)

package.save(output_directory=package_output_directory)

2020/10/30 00:02:15 Downloading source code...
2020/10/30 00:02:17 Finished downloading source code
2020/10/30 00:02:17 Creating Docker network: acb_default_network, driver: 'bridge'
2020/10/30 00:02:18 Successfully set up Docker network: acb_default_network
2020/10/30 00:02:18 Setting up Docker configuration...
2020/10/30 00:02:18 Successfully set up Docker configuration
2020/10/30 00:02:18 Logging in to registry: 6582efd0543e4e738aaffac05ae1cb8f.azurecr.io
2020/10/30 00:02:19 Successfully logged into 6582efd0543e4e738aaffac05ae1cb8f.azurecr.io
2020/10/30 00:02:19 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/10/30 00:02:19 Scanning for dependencies...
2020/10/30 00:02:20 Successfully scanned dependencies
2020/10/30 00:02:20 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB
Step 1/15 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200723.v1@sha256:6446903197a22de6711cb

In [4]:
acr=package.get_container_registry()

acr_address = acr.address
acr_username = acr.username
acr_password = acr.password

In [10]:
!docker login $acr_address -u $acr_username -p $acr_password

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/azureuser/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
!docker build --tag $model_name $package_output_directory

Sending build context to Docker daemon  41.47kB
Step 1/6 : FROM 6582efd0543e4e738aaffac05ae1cb8f.azurecr.io/azureml/azureml_5ec37a853586b44badb12b24c66a563c
latest: Pulling from azureml/azureml_5ec37a853586b44badb12b24c66a563c

8bd67045: Pulling fs layer 
e4862c05: Pulling fs layer 
8949dcb1: Pulling fs layer 
61589e70: Pulling fs layer 
f5a52dc7: Pulling fs layer 
0fd3d326: Pulling fs layer 
d81a8c64: Pulling fs layer 
36e3f3d3: Pulling fs layer 
f55ccd20: Pulling fs layer 
dc9414f3: Pulling fs layer 
aa572370: Pulling fs layer 
93c79c2d: Pulling fs layer 
6b63e60e: Pulling fs layer 
692dc7e2: Pulling fs layer 
e06e4c5a: Pulling fs layer 
2ef9f9ae: Pulling fs layer 
8cfeaeca: Pulling fs layer 
Digest: sha256:c92172acbe455a6c2134e7a4e419a5807c141dc02a925f4f7e8d6d8806165f882K
Status: Downloaded newer image for 6582efd0543e4e738aaffac05ae1cb8f.azurecr.io/azureml/azureml_5ec37a853586b44badb12b24c66a563c:latest
 ---> 820b22676e83
Step 2/6 : ENV AZUREML_MODEL_DIR=azureml-models/tempanomaly/

In [ ]:
!docker run -p 6789:5001 --rm $model_name

2020-10-30T00:12:34,998182360+00:00 - gunicorn/run 
2020-10-30T00:12:35,009221843+00:00 - iot-server/run 
2020-10-30T00:12:35,009274444+00:00 - rsyslog/run 
2020-10-30T00:12:35,011010189+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2020-10-30T00:12:35,081116782+00:00 - iot-server/finish 1 0
2020-10-30T00:12:35,082320713+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (11)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 42
SPARK_HOME not set. Skipping PySpark Initialization.
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.17.0 (/azureml-envs/azureml_917177c5fac4333b4d877fb8071f0b56/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.15.0')).
Failure while loading azureml_run_type_providers. Failed to loa

In [ ]:
!docker ps